In [31]:
from openpyxl import load_workbook

In [32]:
wb2 = load_workbook('./data/Du_fri/上樓梯測試.xlsx')

In [33]:
for sheet in wb2.get_sheet_names():
    sheet_ranges = wb2['1']

In [34]:
sheet_ranges

<Worksheet "1">

In [35]:
print sheet_ranges['A6'].value

4


In [36]:
import xlrd

In [37]:
import xlrd
import csv

In [38]:
workbook = xlrd.open_workbook('./data/Du_fri/上樓梯測試.xlsx')

In [39]:
name_test = workbook.sheet_names()[0]

In [40]:
worksheet = workbook.sheet_by_name(name_test)

In [41]:
csvfile = open('test.csv', 'wb')

In [42]:
wr = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

In [43]:
for rownum in xrange(worksheet.nrows):
    wr.writerow(
        list(x.encode('utf-8') if type(x) == type(u'') else x for x in worksheet.row_values(rownum)))

In [44]:
import pandas as pd

In [48]:
dataset = pd.read_csv('./test.csv', skiprows=1, index_col='Number')

In [52]:
dataset[dataset['Time(s)'].st]

,Time(s),X(g),Y(g),Z(g),R(g),Theta(deg),Phi(deg)
Number,,,,,,,


In [53]:
test = dataset['Time(s)']

In [ ]:
test.

In [47]:
dataset

,Number,Time(s),X(g),Y(g),Z(g),R(g),Theta(deg),Phi(deg)
0,1.0,10.062000,-0.046859,0.893250,0.251867,0.929262,74.273840,93.002930
1,2.0,10.062000,-0.043930,0.926441,0.246010,0.959554,75.144680,92.714836
2,3.0,10.074000,-0.050764,0.918631,0.257724,0.955449,74.351100,93.162970
3,4.0,10.100000,-0.029287,0.920584,0.293845,0.966787,72.305620,91.822160
4,5.0,10.135000,-0.044907,0.931322,0.270415,0.970826,73.826810,92.760550
5,6.0,10.135000,-0.048811,0.927417,0.250891,0.961994,74.882286,93.012790
6,7.0,10.167001,-0.014643,0.984039,0.254796,1.016596,75.484840,90.852554
7,8.0,10.178000,-0.054669,0.932299,0.285059,0.976436,73.025986,93.355910
8,9.0,10.201000,-0.084932,0.956704,0.292869,1.004126,73.042320,95.073166
9,10.0,10.274000,-0.085908,0.966467,0.229414,0.997030,76.697205,95.079605
